This is a page for exploring a single STAC collection

In [ ]:
import geoviews as gv
import panel as pn
import pystac

import utils.cards
import utils.location
import utils.template
import utils.xyt
import utils.zarr

pn.extension()
gv.extension("bokeh")

In [ ]:
query_params = utils.location.UrlQueryParams()

In [ ]:
# In a notebook environment pn.state.location is not initialized until the first plot has been displayed

pn.state.location.sync(query_params, {"site_id": "site-id", "collection_id": "collection-id"})  # type: ignore

In [ ]:


def get_collection_content(collection: pystac.Collection):
    extent = utils.xyt.Extent.from_pystac(collection.extent)
    xyt = utils.xyt.XYT(extent=extent)
    obj = utils.zarr.ZarrDataset.from_pystac(xyt, collection)
    crs_txt = pn.pane.Markdown(
        f"""
        The controls below allow you to explore this datacube in time and space.
        You can enter a point of interest (latitude and longitude) in the form provided,
        or click on the map to select a point.
        You can also select a date of interest using the date slider,
        or by clicking on the time series plot.
        
        The map next to the point of interest controls is shown in the web mercator projection.
        However, this collection is in the EPSG:{obj.crs.to_epsg()} coordinate reference system,
        and is plotted in that projection below.
        """
    )
    return [crs_txt, xyt, obj]

In [ ]:
def get_content(site: pystac.Catalog | None, collection: pystac.Collection | None) -> pn.Column:
    elements = []

    if site is None:
        elements.append("Site not found")
    elif collection is None:
        elements.append("Collection not found")
    else:
        site_name = site.title or site.id
        collection_name = collection.title or collection.id
        elements.append(
            pn.pane.Markdown(
                f"""
                # Data explorer

                On this page you can interactively explore our data collection {collection_name}
                for the site {site_name}.
                """
            )
        )
        elements.append(pn.pane.Markdown("## Site"))
        elements.append(utils.cards.site(site, collapsed=True))
        elements.append(pn.pane.Markdown("## Collection"))
        elements.append(utils.cards.collection(site, collection, collapsed=True))
        elements.append(pn.pane.Markdown("## Data"))
        elements.extend(get_collection_content(collection))

    return pn.Column(
        *elements,
        align=("start", "center"),
        min_width=400,
        max_width=1200,
        sizing_mode="stretch_width",
    )

In [ ]:
content = pn.bind(
    get_content,
    site=query_params.param.site,
    collection=query_params.param.collection,
)

In [ ]:
template = utils.template.get_template(content)
template.servable();